In [1]:
!pip install --upgrade --quiet langchain
#!pip install --upgrade --quiet langchain   # This line is commented out.
!pip install --quiet langchain-community
!pip install --upgrade --quiet langchain-together
!pip install youtube_transcript_api
!pip install pytube


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.9 MB/s eta 0:00:00


In [6]:
from langchain_together import ChatTogether
import api_key

llm = ChatTogether(api_key=api_key.api,temperature=0.0,
                   model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

In [8]:
pip show youtube-transcript-api


Name: youtube-transcript-api
Version: 1.0.3
Summary: This is an python API which allows you to get the transcripts/subtitles for a given YouTube video. It also works for automatically generated subtitles, supports translating subtitles and it does not require a headless browser, like other selenium based solutions do!
Home-page: https://github.com/jdepoix/youtube-transcript-api
Author: Jonas Depoix
Author-email: jonas.depoix@web.de
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: defusedxml, requests
Required-by: 


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_community.document_loaders import YoutubeLoader

video_url = 'https://www.youtube.com/watch?v=ssmlfH0bmi0&ab_channel=ShaneHummus'
video_id = YoutubeLoader.extract_video_id(video_url)

transcript = YouTubeTranscriptApi.get_transcript(video_id)

# Combine all transcript pieces into a single string
full_transcript = " ".join([entry['text'] for entry in transcript])

# Create a Document object with the transcript
from langchain_core.documents import Document
data = [Document(page_content=full_transcript, metadata={"video_id": video_id})]

print(f"Video ID: {video_id}")
print(f"Transcript data: {data}")



Video ID: ssmlfH0bmi0
Transcript data: [Document(metadata={'video_id': 'ssmlfH0bmi0'}, page_content="today we're going to be diving into something that might just blow your mind imagine getting a Harvard Education for free no this isn't a glitch in The Matrix and no you don't need a time machine or a small fortune we're talking about five incredible free online courses from Harvard that could Skyrocket your careers faster than you can say Veritas so whether you're a tech Enthusiast a data Dynamo or a Wordsmith in the making there's something here for you but before we dive in let's address the elephant in the room yes these are courses that are actually from Harvard yes they're completely free and no you don't need to be a genius to take them all you need is curiosity and the willingness to learn so buckle up grab your favorite note ticking tool and let's embark on this ivy league Adventure kicking off our list is the crown jewel of Harvard's online offerings which is the cs50 introduc

In [3]:
data[0].page_content

"today we're going to be diving into something that might just blow your mind imagine getting a Harvard Education for free no this isn't a glitch in The Matrix and no you don't need a time machine or a small fortune we're talking about five incredible free online courses from Harvard that could Skyrocket your careers faster than you can say Veritas so whether you're a tech Enthusiast a data Dynamo or a Wordsmith in the making there's something here for you but before we dive in let's address the elephant in the room yes these are courses that are actually from Harvard yes they're completely free and no you don't need to be a genius to take them all you need is curiosity and the willingness to learn so buckle up grab your favorite note ticking tool and let's embark on this ivy league Adventure kicking off our list is the crown jewel of Harvard's online offerings which is the cs50 introduction to computer science now I know what some of you might be thinking computer science isn't that j

In [7]:
messages = [
    (
        "system",
        """Read through the entire transcript carefully.
           Provide a concise summary of the video's main topic and purpose.
           Extract and list the five most interesting or important points from the transcript. For each point: State the key idea in a clear and concise manner.

        - Ensure your summary and key points capture the essence of the video without including unnecessary details.
        - Use clear, engaging language that is accessible to a general audience.
        - If the transcript includes any statistical data, expert opinions, or unique insights, prioritize including these in your summary or key points.""",
    ),
    ("human", data[0].page_content),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="**Summary:**\n\nThe video discusses five free online courses from Harvard University that can help individuals boost their careers. The courses cover various topics, including computer science, data science, digital humanities, persuasive writing and public speaking, and artificial intelligence. These courses are designed to be accessible to beginners and can help individuals develop valuable skills that are in high demand in the job market.\n\n**Key Points:**\n\n1. **CS50 Introduction to Computer Science**: This course is a comprehensive introduction to computer science, covering topics such as programming languages, data structures, and algorithms. It's designed for beginners and can help individuals develop a problemsolving mindset that can be applied to any field.\n\n2. **R Basics and Data Science**: This course teaches the basics of the R programming language and data science, covering topics such as data types, vectors, and data visualization. It's a great cour

In [8]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

# Define a PromptTemplate for summarizing video transcripts
# The template includes instructions for the AI model on how to process the transcript
product_description_template = PromptTemplate(
    input_variables=["video_transcript"],
    template="""
    Read through the entire transcript carefully.
           Provide a concise summary of the video's main topic and purpose.
           Extract and list the five most interesting or important points from the transcript.
           For each point: State the key idea in a clear and concise manner.

        - Ensure your summary and key points capture the essence of the video without including unnecessary details.
        - Use clear, engaging language that is accessible to a general audience.
        - If the transcript includes any statistical data, expert opinions, or unique insights,
        prioritize including these in your summary or key points.

    Video transcript: {video_transcript}    """
)

In [9]:
## invoke the chain with the video transcript
chain = LLMChain(llm=llm, prompt=product_description_template)

# Run the chain with the provided product details
summary = chain.invoke({
    "video_transcript": data[0].page_content
})

<ipython-input-9-f9de8f4629dd>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=product_description_template)


In [10]:
summary['text']


"**Summary:**\n\nThe video discusses five free online courses from Harvard University that can help individuals boost their careers. The courses cover various topics, including computer science, data science, digital humanities, persuasive writing and public speaking, and artificial intelligence. These courses are designed to be accessible to beginners and can help individuals develop valuable skills that can be applied to various fields.\n\n**Key Points:**\n\n1. **CS50 Introduction to Computer Science**: This course is a comprehensive introduction to computer science, covering topics such as programming languages, data structures, and algorithms. It's designed for beginners and has been taken by over 3.7 million people worldwide.\n\nKey Idea: CS50 is a beginner-friendly course that can help individuals develop a problemsolving mindset and computational thinking skills.\n\n2. **R Basics for Data Science**: This course teaches the basics of R programming language, which is used for data

In [11]:
from IPython.display import Markdown, display

display(Markdown(summary['text']))

**Summary:**

The video discusses five free online courses from Harvard University that can help individuals boost their careers. The courses cover various topics, including computer science, data science, digital humanities, persuasive writing and public speaking, and artificial intelligence. These courses are designed to be accessible to beginners and can help individuals develop valuable skills that can be applied to various fields.

**Key Points:**

1. **CS50 Introduction to Computer Science**: This course is a comprehensive introduction to computer science, covering topics such as programming languages, data structures, and algorithms. It's designed for beginners and has been taken by over 3.7 million people worldwide.

Key Idea: CS50 is a beginner-friendly course that can help individuals develop a problemsolving mindset and computational thinking skills.

2. **R Basics for Data Science**: This course teaches the basics of R programming language, which is used for data analysis and visualization. It's part of Harvard's professional certificate in data science program and can help individuals become data-driven professionals.

Key Idea: R is a versatile and powerful language that can help individuals analyze and visualize data, and it's a valuable skill in today's job market.

3. **Digital Humanities**: This course explores the intersection of technology and humanities, covering topics such as digital tools, data visualization, and text analysis. It's designed to help individuals become cultural codebreakers and develop skills that can be applied to various fields.

Key Idea: Digital humanities is a field that combines technology and humanities, and it can help individuals develop skills that are valuable in today's job market.

4. **The Art of Persuasive Writing and Public Speaking**: This course teaches the principles of rhetoric, which can help individuals become effective communicators and persuasive writers. It's designed to help individuals develop skills that can be applied to various fields, including business, politics, and education.

Key Idea: Rhetoric is a valuable skill that can help individuals become effective communicators and persuasive writers, and it's a must-have in today's job market.

5. **Introduction to Artificial Intelligence with Python**: This course teaches the basics of artificial intelligence, covering topics such as search algorithms, knowledge representation, and machine learning. It's designed to help individuals become AI wizards and develop skills that can be applied to various fields.

Key Idea: AI is a rapidly growing field that can help individuals develop skills that are valuable in today's job market, and it's not just about creating smart machines, but also solving complex problems in innovative ways.